In [1]:
import os
print "current dir: ", os.getcwd()
os.chdir("/home/foodmap/food101/")
print "current dir: ", os.getcwd()

current dir:  /home/foodmap/food101/notebooks
current dir:  /home/foodmap/food101


In [2]:
from location.locations import Cities, Countries
from twitter.Tweet import Tweet
from location.get_location_from_tweet import *
import random

citiesIndex, citiesInfo = Cities.loadFromFile()
countriesIndex, countriesInfo = Countries.loadFromFile()
ccDict = Countries.countryCodeDict(countriesInfo)

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [7]:
inputFile = "/home/foodmap/data/food-tweets/food-tweets-2017-06-15.json.gz"

In [8]:
def get_media_url(tweet):
    if "entities" in tweet:
        if "media" in tweet["entities"]:
            foundMedia = tweet["entities"]["media"]
            return foundMedia[0]["media_url"]
        else:
            return None
    else:
        print tweet
        

In [9]:
def get_food_category(img_url):
    categ_list = ["Pizza", "Pasta", "Sushi", None]
    return random.choice(categ_list)

In [10]:
import codecs, json
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
countImg = 0
countIndexEntries = 0
countCategory = 0
countPlace = 0

with codecs.open("locationresults-no-ambiguity.txt", "w", "utf-8") as f:
    for tweet in tweetsAsDict:
        i+=1
        hasImg = False
        hasCategory = False
        hasCountry = False

        # 1. has photo?
        if get_media_url(tweet) is not None:
            hasImg = True
            countImg += 1
        else:
            continue

        # 2. has food category?
#         if get_food_category(tweet) is not None:
#             hasCategory = True
#             countCategory += 1
#         else:
#             continue

        # 3. has location? we need city and country; record 2 separate fields
        ## 3.1. check for place
        tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code, user_loc = getLocationData(tweet)
        #print tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code
        ## 3.2. check for user location
        if (tweet_place_country is None):
            user_cities, user_countries = getUserLocation(user_loc, citiesIndex, citiesInfo, countriesIndex, countriesInfo)
            inferred_countries = inferCountryFromCity(user_cities, citiesIndex, citiesInfo, ccDict)
            city, country = getFinalUserLocation(user_cities, user_countries, inferred_countries)
        else:
            city = tweet_place_city
            country = tweet_place_country

        if user_loc is not None:
            f.write("User_loc:" + json.dumps(user_loc) + "\n")
            f.write("If Place: " + json.dumps(tweet_place_city) + "\t" + json.dumps(tweet_place_country) + "\n")
            f.write("Final Loc: " + json.dumps(city) + "\t" + json.dumps(country) + "\n")
            f.write("\n")

        if (tweet_place_country is not None) or (country is not None):
            hasCountry =True
            countPlace += 1

        # SEE if CONDITIONS are met
        if hasImg and hasCategory and hasCountry:
            countIndexEntries +=1

print "Total relevant tweets in a day: ", i
print "Tweets with media", countImg
print "Tweets with category", countCategory
print "Tweets with place", countPlace
print "Tweets in index", countIndexEntries

Opening file:  /home/foodmap/data/food-tweets/food-tweets-2017-06-15.json.gz


KeyError: 'entities'